Required dependecies

In [ ]:
!pip install --upgrade pip
!pip install --upgrade tensorflow
!pip install numpy --upgrade
!pip install -q kaggle
!pip install zipfile36

In [2]:
!mkdir checkpoints
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d moltean/fruits

 99% 1.28G/1.28G [00:42<00:00, 65.5MB/s]
100% 1.28G/1.28G [00:43<00:00, 32.0MB/s]


In [3]:
import zipfile
with zipfile.ZipFile('fruits.zip', 'r') as zipObj:
   zipObj.extractall()

# **Extraction of types from "fruits-360" dataset**

In [4]:
!mkdir fruits-360-type
!mkdir fruits-360-type/Test
!mkdir fruits-360-type/Training

import shutil
import os

soruce_dir_test="fruits-360_dataset/fruits-360/Test/"
soruce_dir_training="fruits-360_dataset/fruits-360/Training/"
test_dir="fruits-360-type/Test/"
train_dir="fruits-360-type/Training/"

def extract_types(directory, destination):
  type_counter={'apple':0,'banana':0,'cherry':0,'grape':0,'peach':0,'pear':0,'pepper':0,'plum':0,'potato':0,'tomato':0}
  for label_type in ['apple','banana','cherry','grape','peach','pear','pepper','plum','potato','tomato']:
    for label_variety in os.listdir(directory):
      if(label_variety.lower().find(label_type)==0 and label_variety.lower().find("grapefruit")!=0):
          image_list=os.listdir(os.path.join(directory,label_variety))
          for image in image_list:
              shutil.copy(os.path.join(directory,label_variety,image), os.path.join(destination,label_type,f"{label_type}_{type_counter[label_type]}.jpg"))
              type_counter[label_type]+=1
      else:
        continue

for label in ['apple','banana','cherry','grape','peach','pear','pepper','plum','potato','tomato']:
  os.system(f"mkdir {os.path.join(test_dir,label)}")
  os.system(f"mkdir {os.path.join(train_dir,label)}")

extract_types(soruce_dir_test, test_dir)
extract_types(soruce_dir_training, train_dir)

#Modules

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# **Classification based on fruit and vegetable types**

In [5]:


train_data=[]
train_label=[]
test_data=[]
test_label=[]
epochs=10
batch_size=16
optimizer="adam"
img_size = (32, 32)
shape = (32, 32, 3)
test_dir = 'fruits-360-type/Test'
train_dir = 'fruits-360-type/Training'
metrics=['accuracy']

labels = os.listdir(train_dir)
num_classes = len(labels)

def calculate_class_weights(train_label):
    train_size=len(train_label)
    weights={}
    for i in range(10):
        weights[i]=0
    for i in train_label:
        weights[i]=weights[i]+1
    for i in range(num_classes):
        weights[i]=train_size/(weights[i]*2.)
    return weights

def load_data(dir):
    i=0
    data=[]
    data_label=[]
    for label in labels:
        images=os.listdir(os.path.join(dir,label))
        for image in images:
            image_path=os.path.join(dir,label,image)
            image=load_img(image_path,color_mode='rgb', target_size=img_size)
            array_image=img_to_array(image)
            array_image=array_image/255.0
            data.append(array_image)
            data_label.append(i)
        i=i+1
    tmp=list(zip(data, data_label))
    data, data_label=zip(*tmp)
    return data, data_label

print("Loading training images...")
train_data, train_label=load_data(train_dir)
train_data=np.array(train_data)
train_label=np.array(train_label)
print("Loading test images...")
test_data, test_label=load_data(test_dir)
test_data=np.array(test_data)
test_label=np.array(test_label)
network=tf.keras.models.Sequential()
network.add(Input(shape=shape))
network.add(Conv2D(32, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(64, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(128, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Flatten())
network.add(Dense(512, activation="softmax"))
network.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=metrics)
print(network.summary())
weights=calculate_class_weights(train_label)
history = network.fit(x=train_data,y=train_label,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1)
train_loss, train_accuracy = network.evaluate(x=train_data,y=train_label,steps=(len(train_data)//batch_size)+1, verbose=1)
test_loss, test_accuracy = network.evaluate(x=test_data, y=test_label,steps=(len(test_data)//batch_size)+1, verbose=1)
print(f"Train accuracy = {train_accuracy}")
print(f"Test accuracy = {test_accuracy}")

Loading training images...
Loading test images...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 max_p

# **Classification based on fruit and vegetable varieties**

In [7]:
train_data=[]
train_label=[]
test_data=[]
test_label=[]
epochs=20
batch_size=16
optimizer="adam"
img_size = (32, 32)
shape = (32, 32, 3)
test_dir = 'fruits-360_dataset/fruits-360/Test'
train_dir = 'fruits-360_dataset/fruits-360/Training'
metrics=['accuracy']

labels = os.listdir(train_dir)
print(labels)
num_classes = len(labels)

def calculate_class_weights(train_label, num_classes):
    train_size=len(train_label)
    weights={}
    for i in range(num_classes):
        weights[i]=0
    for i in train_label:
        weights[i]=weights[i]+1
    for i in range(num_classes):
        weights[i]=train_size/(weights[i]*2.)
    return weights

def load_data(dir, labels):
    i=0
    data=[]
    data_label=[]
    for label in labels:
        images=os.listdir(os.path.join(dir,label))
        for image in images:
            image_path=os.path.join(dir,label,image)
            image=load_img(image_path,color_mode='rgb', target_size=img_size)
            array_image=img_to_array(image)
            array_image=array_image/255.0
            data.append(array_image)
            data_label.append(i)
        i=i+1
    tmp=list(zip(data, data_label))
    data, data_label=zip(*tmp)
    return data, data_label

print("Loading training images...")
train_data, train_label=load_data(train_dir, labels)
train_data=np.array(train_data)
train_label=np.array(train_label)
print("Loading test images...")
test_data, test_label=load_data(test_dir, labels)
test_data=np.array(test_data)
test_label=np.array(test_label)
network=tf.keras.models.Sequential()
network.add(Input(shape=shape))
network.add(Conv2D(32, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Conv2D(64, (3, 3), padding="same", activation="relu", strides=(1, 1)))
network.add(MaxPooling2D((2, 2), strides=(2, 2)))
network.add(Flatten())
network.add(Dense(512, activation="relu"))
network.add(Dropout(0.5))
network.add(Dense(num_classes, activation="softmax"))
network.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=metrics)
print(network.summary())
weights=calculate_class_weights(train_label, num_classes)
history = network.fit(x=train_data,y=train_label,
                        batch_size=batch_size,
                        epochs=epochs,
                        class_weight=weights,
                        verbose=1)
train_loss, train_accuracy = network.evaluate(x=train_data,y=train_label, verbose=1)
test_loss, test_accuracy = network.evaluate(x=test_data, y=test_label, verbose=1)
predicted=network.predict(test_data, batch_size=16, verbose=1)
predicted = predicted.argmax(axis=-1)
print(f"Train accuracy = {train_accuracy}")
print(f"Test accuracy = {test_accuracy}")

['Kiwi', 'Melon Piel de Sapo', 'Eggplant', 'Pear Williams', 'Chestnut', 'Cherry Wax Black', 'Huckleberry', 'Pear Abate', 'Clementine', 'Pear Red', 'Lychee', 'Cocos', 'Mango', 'Granadilla', 'Avocado', 'Corn Husk', 'Peach Flat', 'Banana Lady Finger', 'Passion Fruit', 'Pear 2', 'Potato Red Washed', 'Nectarine', 'Orange', 'Cherry Wax Yellow', 'Cauliflower', 'Maracuja', 'Mandarine', 'Nut Forest', 'Mango Red', 'Apple Golden 3', 'Apple Braeburn', 'Apple Red Yellow 1', 'Tomato 1', 'Watermelon', 'Pitahaya Red', 'Kaki', 'Cherry 2', 'Strawberry Wedge', 'Guava', 'Tomato Heart', 'Corn', 'Nut Pecan', 'Peach 2', 'Limes', 'Banana Red', 'Tomato Yellow', 'Grape White 4', 'Grape Blue', 'Pineapple Mini', 'Pear Stone', 'Grape White 3', 'Tomato not Ripened', 'Tangelo', 'Potato Red', 'Ginger Root', 'Kohlrabi', 'Onion Red Peeled', 'Pepper Yellow', 'Plum 2', 'Pepper Red', 'Walnut', 'Apricot', 'Onion Red', 'Pear', 'Raspberry', 'Cucumber Ripe', 'Cactus fruit', 'Lemon Meyer', 'Tomato 4', 'Pepino', 'Cucumber Ripe 